# Loading DataSets & Libraries.

In [53]:
import numpy as np
import pandas as pd

In [54]:
df = pd.read_csv('/content/train.csv')

In [55]:
df.head(4)

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S

In [56]:
df.drop(columns= ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace= True)

# train_test_split()

In [57]:
from sklearn.model_selection import train_test_split
x = df.drop(columns= ['Survived'])
y = df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 42)
x_train.shape, x_test.shape

((712, 7), (179, 7))

In [58]:
x_train.sample()

Pclass     Sex  Age  SibSp  Parch  Fare Embarked
612       3  female  NaN      1      0  15.5        Q

# ColumnTransformer

In [59]:
# 1-
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]),
    ('impute_embarked', SimpleImputer(strategy= 'most_frequent'),[6])
], remainder= 'passthrough')

In [60]:
# 2-
from sklearn.preprocessing import OneHotEncoder

trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output= False, handle_unknown= 'ignore'), [1, 6])
], remainder= 'passthrough')

In [61]:
# 3-
from sklearn.preprocessing import MinMaxScaler

trf3 = ColumnTransformer([
    ('scaling', MinMaxScaler(), slice(0,10))
], remainder= 'drop')

In [62]:
# 4-
from sklearn.feature_selection import SelectKBest, chi2

trf4 = SelectKBest(score_func= chi2, k= 8)

In [63]:
# 5-
from sklearn.tree import DecisionTreeClassifier
trf5 = DecisionTreeClassifier()

# Pipelines()

In [64]:
from sklearn.pipeline import Pipeline, make_pipeline

pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

# **make_pipeline()**

In [66]:
pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [67]:
pipe.fit(x_train, y_train) # If you're traning a model then you use fit otherwise you should use fit_transform

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x79f413c8dda0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [68]:
y_pred = pipe.predict(x_test)

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6256983240223464